<a href="https://colab.research.google.com/github/saurograndi/nlp-t5-summarizer/blob/main/mt5-it-wits-summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization with multilingual T5 (mT5)

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install datasets
!pip install nltk
!pip install rouge_score
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d54a0390472d2437e1bcff6b30034132cc4535a9436cd7b0e584665bff907a49
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('token'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import datasets

wits = datasets.load_dataset("it5/datasets", "wits")
wits

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['summary', 'source'],
        num_rows: 679426
    })
    valid: Dataset({
        features: ['summary', 'source'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['summary', 'source'],
        num_rows: 10000
    })
})

In [ ]:
def get_samples(dataset, num_samples=100, seed=42):
    train_sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    valid_sample = dataset["valid"].shuffle(seed=seed).select(range(15))
    test_sample = dataset["test"].shuffle(seed=seed).select(range(15))
    return datasets.DatasetDict({"train":train_sample, "valid":valid_sample, "test":test_sample})

In [ ]:
wits_small = get_samples(wits)
wits_small

DatasetDict({
    train: Dataset({
        features: ['summary', 'source'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['summary', 'source'],
        num_rows: 15
    })
    test: Dataset({
        features: ['summary', 'source'],
        num_rows: 15
    })
})

In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Source: {example['source']}'")
        print(f"\n'>> Summary: {example['summary']}'")


show_samples(wits, num_samples=1)


'>> Source: Alla fine del 1530 Mōri Motonari, inizialmente alleato del clan Amago (noto anche come Amako), si sganciò da quest'ultimi e si alleò con gli Ōuchi. Approfittando della crescente debolezza del clan Takeda di Aki, Motonari divenne sempre più potente nella provincia. Nel 1540 il vecchio daimyō degli Amago, Tsunehisa, si ritirò nominalmente e consegnò la guida del clan a suo nipote, Haruhisa (noto anche come Akihisa).  In quell'anno Haruhisa concepì un piano per distruggere Motonari e portare la provincia di Aki sotto il controllo degli Amago. Quando un consiglio dei servitori di Amago fu chiamato a discutere della pianificazione della campagna, quasi tutti parlarono a favore dell'attacco. Amago Hisayuki, tuttavia, considerò i rischi troppo grandi e si dichiarò contrario, ma fu deriso da Tsunehisa e marcato come un codardo. A Hisayuki fu affidato il compito di ingaggiare un alleato dei Mōri, il clan Shishido di Aki, come parte di un'operazione iniziale e concorrente della più 

In [ ]:
#max_input_length = 69632
#max_target_length = 5120
max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["source"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets_small = wits_small.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets_small["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets_small["valid"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, epochs=8, verbose=2
)

Epoch 1/8
12/12 - 3s - loss: 13.6571 - val_loss: 8.9381 - 3s/epoch - 290ms/step
Epoch 2/8
12/12 - 3s - loss: 14.7661 - val_loss: 8.9381 - 3s/epoch - 260ms/step
Epoch 3/8
12/12 - 3s - loss: 14.6897 - val_loss: 8.9381 - 3s/epoch - 242ms/step
Epoch 4/8
12/12 - 3s - loss: 15.9462 - val_loss: 8.9381 - 3s/epoch - 253ms/step
Epoch 5/8
12/12 - 3s - loss: 14.7893 - val_loss: 8.9381 - 3s/epoch - 265ms/step
Epoch 6/8
12/12 - 3s - loss: 14.1575 - val_loss: 8.9381 - 3s/epoch - 240ms/step
Epoch 7/8
12/12 - 3s - loss: 14.1784 - val_loss: 8.9381 - 3s/epoch - 252ms/step
Epoch 8/8
12/12 - 3s - loss: 15.1120 - val_loss: 8.9381 - 3s/epoch - 269ms/step
